In [1]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Define the controlled vocabularies and generated tags
controlled_vocab = {
    'adults_selfie': ['Group photo', 'friends', 'smiling', 'diverse group'],
    'baby_girl': ['Infant', 'toddler', 'child', 'cute', 'blue eyes', 'headband'],
    'black_girl': ['Portrait', 'serious expression', 'freckles', 'black hair'],
    'black_portrait': ['Close-up', 'serious expression', 'dark skin', 'natural look'],
    'elderly_doing_exercise': ['Seniors', 'stretching', 'group activity', 'indoor'],
    'elderly_gardening': ['Seniors', 'outdoor activity', 'gardening', 'watering plants'],
    'elderly_playing_table_tennis': ['Seniors', 'outdoor activity', 'table tennis', 'happy'],
    'elderly_woman_gardening_in_backyard_with_daughter': ['Seniors', 'family', 'gardening', 'wheelchair'],
    'family_photos': ['Family', 'beach', 'casual', 'black and white'],
    'gladiators_pose_for_tourists_in_rome': ['Costumes', 'tourists', 'historical reenactment', 'group'],
    'group_of_people_mixed': ['Diverse group', 'portrait', 'serious expressions', 'different ages'],
    'kid_with_cows': ['Baby', 'animals', 'farm', 'outdoors', 'cute'],
    'kids_playing_football': ['Children', 'sports', 'soccer', 'football', 'park'],
    'kids_playing_in_park': ['Children', 'outdoors', 'friends', 'happy'],
    'kids_playing_outdoors': ['Children', 'nature', 'happiness', 'diversity'],
    'people_eating_pizza': ['Friends', 'dining', 'food', 'casual gathering'],
    'twin_babies': ['Infants', 'sleeping', 'siblings', 'cozy'],
    'white_adult': ['Portrait', 'close-up', 'natural', 'serious expression'],
    'white_kid': ['Boy', 'portrait', 'smiling', 'casual attire'],
    'white_kids_playground': ['Group photo', 'children', 'playground', 'school'],
}


azure_tags = {
    'adults_selfie': ['person', 'smile', 'human face', 'clothing', 'man', 'happy', 'friendship', 'laugh', 'fun', 'tooth', 'selfie', 'adolescent', 'group', 'love', 'social group', 'youth', 'posing', 'people', 'smiling', 'woman', 'indoor'],
    'baby_girl': ['toddler', 'human face', 'person', 'baby', 'clothing', 'child', 'baby & toddler clothing', 'child model'],
    'black_girl': ['human face', 'person', 'lip', 'eyebrow', 'eyelash', 'skin', 'portrait photography', 'cheek', 'portrait', 'girl', 'lipstick', 'forehead', 'woman', 'face'],
    'black_portrait': ['human face', 'person', 'eyebrow', 'skin', 'eyelash', 'cheek', 'forehead', 'lip', 'portrait', 'indoor', 'face'],
    'elderly_doing_exercise': ['clothing', 'person', 'wall', 'furniture', 'indoor', 'chair', 'footwear', 'man', 'woman', 'group', 'people', 'adult', 'dance', 'floor'],
    'elderly_gardening': ['clothing', 'person', 'outdoor', 'plant', 'grass', 'gardener', 'farmworker', 'gardening', 'footwear', 'woman', 'people', 'fall'],
    'elderly_playing_table_tennis': ['person', 'table-tennis table', 'clothing', 'outdoor', 'table', 'tree', 'man', 'ping pong', 'table tennis racket', 'grass', 'standing'],
    'elderly_woman_gardening_in_backyard_with_daughter': ['person', 'clothing', 'outdoor', 'wheel', 'plant', 'sun hat', 'flower', 'wheelchair', 'grass', 'woman', 'sitting', 'hat', 'lady', 'girl'],
    'family_photos': ['outdoor', 'sky', 'person', 'clothing', 'people', 'man', 'ground', 'water', 'beach', 'group', 'black and white', 'standing'],
    'gladiators_pose_for_tourists_in_rome': ['clothing', 'person', 'man', 'footwear', 'woman', 'costume', 'cosplay', 'fan convention', 'fashion accessory', 'people', 'outdoor', 'ground', 'street', 'armor'],
    'group_of_people_mixed': ['human face', 'woman', 'clothing', 'person', 'fashion'],
    'kid_with_cows': ['grass', 'outdoor', 'plant', 'human face', 'field', 'farm', 'person', 'girl', 'child'],
    'kids_playing_football': ['person', 'outdoor', 'grass', 'tree', 'football', 'ball', 'athletic game', 'sport', 'footwear', 'soccer ball', 'soccer', 'soccer player', 'playground', 'sports equipment', 'playing', 'field', 'football player', 'kick', 'forward', 'young', 'group', 'boy', 'child'],
    'kids_playing_in_park': ['clothing', 'person', 'human face', 'tree', 'outdoor', 'girl', 'smile', 'toddler', 'group', 'sitting', 'young', 'people', 'child', 'boy', 'ground', 'plant'],
    'kids_playing_outdoors': ['person', 'grass', 'outdoor', 'clothing', 'football', 'soccer', 'building', 'footwear', 'ball', 'plant', 'boy', 'man', 'jeans', 'playground', 'people', 'girl', 'woman'],
    'people_eating_pizza': ['person', 'clothing', 'human face', 'food', 'man', 'people', 'table', 'woman', 'group', 'tableware', 'fast food', 'supper', 'lunch', 'eating', 'indoor', 'sitting', 'wall', 'dinner', 'meal', 'banquet'],
    'twin_babies': ['newborn', 'person', 'human face', 'child', 'toddler', 'boy', 'indoor', 'baby', 'sleep', 'baby & toddler clothing', 'comfort', 'skin', 'nap', 'wall'],
    'white_adult': ['human face', 'person', 'skin', 'portrait photography', 'eyebrow', 'lip', 'portrait', 'eyelash', 'lady', 'layered hair', 'brown hair', 'cheek', 'chin', 'throat', 'forehead', 'female person', 'woman', 'neck', 'lipstick', 'long hair', 'girl', 'face'],
    'white_kid': ['human face', 'person', 'smile', 'clothing', 'chin', 'neck', 'skin', 'portrait', 'bangs', 'portrait photography', 'lip', 'toddler', 'sleeve', 'wall', 'young', 'boy', 'shirt', 'child'],
    'white_kids_playground': ['clothing', 'outdoor', 'person', 'footwear', 'tree', 'group', 'playground', 'people', 'grass', 'boy', 'child', 'young', 'park', 'red', 'climbing frame'],
}

rekognition_tags = {
    'adults_selfie': ['people', 'person', 'head', 'face', 'happy', 'smile', 'laughing', 'adult', 'bride', 'woman'],
    'baby_girl': ['person', 'sitting', 'face', 'head', 'photography', 'portrait', 'baby', 'happy', 'finger', 'sleeve'],
    'black_girl': ['head', 'person', 'face', 'photography', 'portrait', 'skin', 'adult', 'female', 'woman', 'freckle'],
    'black_portrait': ['face', 'head', 'person', 'photography', 'portrait', 'skin', 'body part', 'neck', 'cosmetics', 'lipstick'],
    'elderly_doing_exercise': ['people', 'person', 'head', 'adult', 'male', 'man', 'face', 'happy', 'celebrating', 'bride'],
    'elderly_gardening': ['garden', 'nature', 'outdoors', 'gardening', 'adult', 'male', 'man', 'person', 'gardener', 'glove'],
    'elderly_playing_table_tennis': ['adult', 'male', 'man', 'person', 'face', 'head', 'ping pong paddle', 'racket', 'sport', 'senior citizen'],
    'elderly_woman_gardening_in_backyard_with_daughter': ['garden', 'nature', 'outdoors', 'gardening', 'adult', 'female', 'person', 'woman', 'hat', 'wheel'],
    'family_photos': ['photography', 'person', 'walking', 'people', 'portrait', 'coat', 'beach', 'pants', 'adult', 'man'],
    'gladiators_pose_for_tourists_in_rome': ['costume', 'person', 'adult', 'female', 'woman', 'male', 'man', 'handbag', 'skirt', 'baby'],
    'group_of_people_mixed': ['photography', 'face', 'head', 'person', 'portrait', 'adult', 'female', 'woman', 'glasses', 'man'],
    'kid_with_cows': ['photography', 'person', 'portrait', 'grass', 'field', 'grassland', 'nature', 'outdoors', 'baby', 'livestock'],
    'kids_playing_football': ['boy', 'child', 'male', 'person', 'female', 'girl', 'people', 'play', 'shorts', 'football'],
    'kids_playing_in_park': ['people', 'person', 'face', 'happy', 'head', 'boy', 'child', 'male', 'female', 'girl'],
    'kids_playing_outdoors': ['soccer ball', 'people', 'person', 'jeans', 'grass', 'boy', 'child', 'male', 'girl', 'teen'],
    'people_eating_pizza': ['brunch', 'people', 'person', 'eating', 'pizza', 'head', 'face', 'adult', 'female', 'woman'],
    'twin_babies': ['blanket', 'photography', 'face', 'head', 'person', 'portrait', 'baby', 'people', 'newborn', 'sleeping'],
    'white_adult': ['face', 'head', 'person', 'photography', 'portrait', 'adult', 'female', 'woman', 'smile', 'skin'],
    'white_kid': ['person', 'photography', 'portrait', 'blonde', 'coat', 'smile', 'jacket', 'boy', 'child', 'male'],
    'white_kids_playground': ['grass', 'people', 'person', 'park', 'shoe', 'female', 'girl', 'teen', 'child', 'male'],
}


vision_tags = {
    'adults_selfie': ['smile', 'nose', 'skin', 'chin', 'mouth', 'facial expression', 'tooth', 'white', 'human', 'neck'],
    'baby_girl': ['face', 'skin', 'arm', 'white', 'product', 'baby & toddler clothing', 'sleeve', 'gesture', 'toddler', 'happy'],
    'black_girl': ['forehead', 'nose', 'brown', 'cheek', 'skin', 'lip', 'eyelash', 'flash photography', 'jaw', 'temple'],
    'black_portrait': ['forehead', 'nose', 'cheek', 'lip', 'eyelash', 'flash photography', 'jaw', 'iris', 'ear', 'no expression'],
    'elderly_doing_exercise': ['furniture', 'gesture', 'happy', 'chair', 'couch', 'event', 'fashion design', 'performing arts', 'fun', 'leisure'],
    'elderly_gardening': ['plant', 'plant community', 'people in nature', 'sky', 'natural environment', 'tree', 'farmer', 'grass', 'agriculture', 'natural landscape'],
    'elderly_playing_table_tennis': ['smile', 'muscle', 'gesture', 'happy', 'leisure', 't-shirt', 'community', 'shorts', 'grass', 'competition event'],
    'elderly_woman_gardening_with_daughter': ['plant', 'flower', 'wheel', 'hat', 'sun hat', 'grass', 'leisure', 'terrestrial plant', 'wheelchair', 'summer'],
    'eledelry_gardening': ['plant', 'natural environment', 'tree', 'sky', 'garden tool', 'people in nature', 'grass', 'rural area', 'landscape', 'compost'],
    'family_photos': ['water', 'people in nature', 'people on beach', 'sky', 'flash photography', 'gesture', 'happy', 'style', 'black-and-white', 'leisure'],
    'gladiators_pose_for_tourists_in_rome': ['footwear', 'shoe', 'hat', 'shorts', 'event', 'belt', 'tradition', 'eyewear', 'costume', 'fun'],
    'group_of_people_mixed': ['glasses', 'skin', 'hairstyle', 'facial expression', 'black', 'fashion', 'flash photography', 'social group', 'smile', 'art'],
    'kid_with_cows': ['nature', 'people in nature', 'happy', 'sunlight', 'rabbit', 'grass', 'hat', 'fawn', 'grassland', 'summer'],
    'kids_playing_football': ['shorts', 'soccer', 'playing sports', 'sports equipment', 'plant', 'football', 'green', 'tree', 'player', 'ball'],
    'kids_playing_in_park': ['face', 'smile', 'plant', 'people in nature', 'botany', 'happy', 'tree', 'leisure', 'grass', 'adaptation'],
    'kids_playing_outdoors': ['jeans', 'shorts', 'plant', 'daytime', 'window', 'vertebrate', 'sports equipment', 'football', 'soccer', 'ball'],
    'people_eating_pizza': ['food', 'tableware', 'human', 'table', 'orange', 'plate', 'sharing', 'cuisine', 'wine glass', 'dish'],
    'twin_babies': ['face', 'cheek', 'skin', 'chin', 'comfort', 'baby', 'gesture', 'happy', 'flash photography', 'toddler'],
    'white_adult': ['nose', 'cheek', 'lip', 'eyebrow', 'eyelash', 'flash photography', 'jaw', 'iris', 'makeover', 'black hair'],
    'white_kid_2': ['nose', 'face', 'smile', 'jaw', 'neck', 'sleeve', 'happy', 'bangs', 'eyelash', 'black hair'],
    'white_kid': ['face', 'chin', 'outerwear', 'smile', 'eye', 'neck', 'sleeve', 'collar', 'happy', 'flash photography'],
    'white_kids_playground': ['plant', 'botany', 'tree', 'leisure', 'red', 'smile', 'grass', 'recreation', 'fun', 'event'],
    'white_lady': ['forehead', 'nose', 'cheek', 'skin', 'lip', 'lipstick', 'eyebrow', 'eyelash', 'eye liner', 'organ']
}


# Function to calculate metrics
def calculate_metrics(controlled_vocab, tags_dict):
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracy_scores = []

    for key in controlled_vocab.keys():
        # Convert both true and predicted tags to lowercase for case-insensitive comparison
        y_true = [1 if tag.lower() in [t.lower() for t in controlled_vocab[key]] else 0 for tag in controlled_vocab[key]]
        y_pred = [1 if tag.lower() in [t.lower() for t in tags_dict.get(key, [])] else 0 for tag in controlled_vocab[key]]

        precision_scores.append(precision_score(y_true, y_pred, zero_division=0))
        recall_scores.append(recall_score(y_true, y_pred, zero_division=0))
        f1_scores.append(f1_score(y_true, y_pred, zero_division=0))
        accuracy_scores.append(accuracy_score(y_true, y_pred))

    return {
        'precision': np.mean(precision_scores),
        'recall': np.mean(recall_scores),
        'f1_score': np.mean(f1_scores),
        'accuracy': np.mean(accuracy_scores)
    }

# Calculate metrics for each AI service
google_metrics = calculate_metrics(controlled_vocab, vision_tags)
azure_metrics = calculate_metrics(controlled_vocab, azure_tags)
rekognition_metrics = calculate_metrics(controlled_vocab, rekognition_tags)

# Print the results
print("Google Cloud Vision Metrics:")
print(google_metrics)
print("\nMicrosoft Azure Metrics:")
print(azure_metrics)
print("\nAmazon Rekognition Metrics:")
print(rekognition_metrics)


Google Cloud Vision Metrics:
{'precision': 0.25, 'recall': 0.06583333333333333, 'f1_score': 0.10285714285714284, 'accuracy': 0.06583333333333333}

Microsoft Azure Metrics:
{'precision': 0.65, 'recall': 0.19666666666666666, 'f1_score': 0.29690476190476195, 'accuracy': 0.19666666666666666}

Amazon Rekognition Metrics:
{'precision': 0.55, 'recall': 0.15499999999999997, 'f1_score': 0.23857142857142857, 'accuracy': 0.15499999999999997}
